In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
torch.set_printoptions(linewidth=140)

# params

In [2]:
input_channels = 1
learning_rate = 0.01
hidden_layer = 100
num_classes = 10
num_epoch = 2
batch_size = 100


# Datas

In [3]:
train_set = torchvision.datasets.FashionMNIST(
    './FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()]) # converting ToTensor
)

# Batch

In [4]:
batch = next(iter(train_set))
images, labels = batch
images.shape, labels

(torch.Size([1, 28, 28]), 9)

In [5]:
# images = images.reshape(-1)

# Model

In [6]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
   
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, kernel_size=2, stride=2)
        
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, kernel_size=2, stride=2)
        
        out = F.relu(self.fc1(out.reshape(-1, 12*4*4)))
        out = F.relu(self.fc2(out))
        
        out = self.out(out)
        return out
    
    
model = CNN()
model  
 

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

# Pass 1 image tensors and get pred

In [7]:
pred_scores = model(images.unsqueeze(0)) # not clear why i am using the unsqueeze. Maybe our model accepts only that
pred_scores

tensor([[ 0.0989, -0.0137, -0.0462, -0.0789,  0.0437,  0.1667, -0.1239, -0.0123, -0.0322,  0.0565]], grad_fn=<AddmmBackward0>)

In [8]:
get_max_pred = pred_scores.argmax(dim=1)
get_max_pred

tensor([5])

# Pass 4 image tensors and get pred

In [12]:
train_loader2 = torch.utils.data.DataLoader(train_set, batch_size=4)
images2, labels2 = next(iter(train_loader2))
images2.shape, labels2

(torch.Size([4, 1, 28, 28]), tensor([9, 0, 0, 3]))

In [15]:

images_pred = model(images2)
images_pred # will get 4 prediction for one img, so We use argMax to get the hieghst i think.

tensor([[ 0.0989, -0.0137, -0.0462, -0.0789,  0.0437,  0.1667, -0.1239, -0.0123, -0.0322,  0.0565],
        [ 0.1046, -0.0199, -0.0160, -0.0756,  0.0526,  0.1758, -0.1128,  0.0258, -0.0414,  0.0696],
        [ 0.1022, -0.0110, -0.0255, -0.0755,  0.0676,  0.1611, -0.1012,  0.0232, -0.0297,  0.0462],
        [ 0.0987, -0.0182, -0.0204, -0.0713,  0.0659,  0.1633, -0.1038,  0.0225, -0.0387,  0.0511]], grad_fn=<AddmmBackward0>)

In [11]:
images_pred.argmax(dim=1)
    

tensor([0, 0, 0, 0])

In [12]:
labels2

tensor([9, 0, 0, 3])

# compare it now

In [13]:
# first way of doing it
compaire = (labels2 == images_pred.argmax(dim=1))
compaire

tensor([False,  True,  True, False])

In [16]:
# second way of doing it
images_pred.argmax(dim=1).eq(labels2)

tensor([False,  True,  True, False])

In [17]:
images_pred.argmax(dim=1).eq(labels2).sum()

tensor(2)